# Highway Vehicle Counting Practice Exercise


In this exercise, you will use the YOLO (You Only Look Once) object detection model to analyze a video of highway traffic.
Your task is to count the number of cars that are leaving the highway (coming toward the camera) on the right side and the number of cars that are joining the road on the left side. The video can be found under `Datasets/Example.mp4`.

## Objectives
- Load and process a video using OpenCV.
- Use the YOLO model to detect vehicles in each frame.
- Use OpenCV to manulate the video.
- Track vehicles as they move through the video frames.
- Count the number of vehicles leaving the highway on the right side.
- Count the number of vehicles joining the road on the left side.

## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
#!pip install ultralytics -q

In [ ]:
import cv2
from ultralytics import YOLO, solutions

## Load the YOLO Model

You will first need to load the YOLO model. You can use a pre-trained YOLO model for this task.
Write the code to load the YOLO model below:


In [ ]:
model = YOLO('yolov8n.pt')

## Prepare the Video vidture

Create a variable to vidture the video frames, you can use `cv2.Videovidture()` to achive this.

In [ ]:
video_path = '/content/Example.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `Videovidture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
mid_line = width // 2
mid_y = height // 2
cross_line = int(height * 0.7)

line_points = [(20, mid_y), (width - 20, mid_y)]
counter = solutions.ObjectCounter(view_img=True,
                                  reg_pts=line_points,
                                  names=model.names,
                                  draw_tracks=True,
                                  line_thickness=2)
print(height,width,fps)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
720 1280 25.03699088856008


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.Videovidture()` to save the video with the bounding boxes and the counted cars on both sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = '/content/YOLO_Exam_output.mp4'
Vid_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width,height))

## Process Video Frames and Identify Vehicles on the Right and Left Sides

For each frame in the video, use the YOLO model to detect and track vehicles. You'll need to write a loop that processes each frame and applies the YOLO model.
In each frame, after detecting the vehicles, determine whether they are on the left or right side of the highway.
You can use the position of the bounding boxes provided by YOLO to do this.
* The video should display bounding boxes around the detected objects.
* The video should display the confidence along side with the object id and class id of each detected and tracked object.
* The video display the number of vehicles on the left side.
* The video display the number of vehicles on the right side.
* The video should display the line in which you counted the objects that have crossed it and counted.

In [ ]:
left_count = 0
right_count = 0
color = (160, 32, 240)
thickness = 2

In [ ]:

left_road_incoming = 0
left_road_outgoing = 0
right_road_incoming = 0
right_road_outgoing = 0

color = (160, 32, 240)
thickness = 2

# Video writer to save the output
Vid_writer = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Track objects in the frame
    tracks = model.track(frame, persist=True, show=False)

    for track in tracks:
        boxes = track.boxes.xyxy
        confidences = track.boxes.conf
        class_ids = track.boxes.cls

        if boxes.shape[0] == 0:
            print("No boxes detected in this frame.")
            continue

        for box, conf, class_id in zip(boxes, confidences, class_ids):
            x1, y1, x2, y2 = map(int, box.cpu().numpy())
            conf = float(conf.cpu().numpy())
            class_id = int(class_id.cpu().numpy())

            box_center_x = (x1 + x2) // 2
            box_center_y = (y1 + y2) // 2

            # Determine which road the vehicle is on
            if box_center_x < mid_line:
                # Vehicle on the left road
                if box_center_y < cross_line:
                    left_road_incoming += 1  # Vehicle entering from the top
                else:
                    left_road_outgoing += 1  # Vehicle exiting towards the bottom
                side = 'Left Road'
            else:
                # Vehicle on the right road
                if box_center_y < cross_line:
                    right_road_incoming += 1  # Vehicle entering from the top
                else:
                    right_road_outgoing += 1  # Vehicle exiting towards the bottom
                side = 'Right Road'

            label = f'{model.names[class_id]}: {conf:.2f}, {side}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

    # Draw the lines
    cv2.line(frame, (0, cross_line), (width, cross_line), (0, 0, 255), 2)  # Horizontal line
    cv2.line(frame, (mid_line, 0), (mid_line, height), (0, 255, 0), 2)  # Vertical line

    # Display counts on the frame
    cv2.putText(frame, f"Left Incoming: {left_road_incoming}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Left Outgoing: {left_road_outgoing}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Right Incoming: {right_road_incoming}", (width - 400, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Right Outgoing: {right_road_outgoing}", (width - 400, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the processed frame to the output video
    Vid_writer.write(frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
Vid_writer.release()
cv2.destroyAllWindows()



0: 384x640 5 cars, 10.3ms
Speed: 2.3ms preprocess, 10.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 6.8ms
Speed: 2.0ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 7.8ms
Speed: 1.9ms preprocess, 7.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 11.6ms
Speed: 2.1ms preprocess, 11.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 7.7ms
Speed: 3.6ms preprocess, 7.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 10.2ms
Speed: 1.8ms preprocess, 10.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 7.3ms
Speed: 2.8ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


## Save and Submit Your Work

In the actual exam you will be asked to submit both the notebook and the output video